# World Bank Data Set Exercise

**About the Dataset:**

_World Bank Projects & Operations provides access to basic information on all of the World Bank's lending projects from 1947 to the present. The dataset includes basic information such as the project title, task manager, country, project id, sector, themes, commitment amount, product line, and financing. It also provides links to publicly disclosed online documents._

**Tasks**

1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [17]:
import json
import pandas as pd
from pandas.io.json import json_normalize

In [163]:
worldbank_df = pd.read_json('data/world_bank_projects.json',convert_dates=['boardapprovaldate'])
print(worldbank_df.columns)
worldbank_df.head()

Index(['_id', 'approvalfy', 'board_approval_month', 'boardapprovaldate',
       'borrower', 'closingdate', 'country_namecode', 'countrycode',
       'countryname', 'countryshortname', 'docty', 'envassesmentcategorycode',
       'grantamt', 'ibrdcommamt', 'id', 'idacommamt', 'impagency',
       'lendinginstr', 'lendinginstrtype', 'lendprojectcost',
       'majorsector_percent', 'mjsector_namecode', 'mjtheme',
       'mjtheme_namecode', 'mjthemecode', 'prodline', 'prodlinetext',
       'productlinetype', 'project_abstract', 'project_name', 'projectdocs',
       'projectfinancialtype', 'projectstatusdisplay', 'regionname', 'sector',
       'sector1', 'sector2', 'sector3', 'sector4', 'sector_namecode',
       'sectorcode', 'source', 'status', 'supplementprojectflg', 'theme1',
       'theme_namecode', 'themecode', 'totalamt', 'totalcommamt', 'url'],
      dtype='object')


,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,{'$oid': '52b213b38594d8a2be17c782'},2014,November,2013-11-01,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,...,TI,IBRD,Active,Y,"{'Percent': 46, 'Name': 'Regional integration'}","[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,{'$oid': '52b213b38594d8a2be17c783'},2014,October,2013-10-31,MIN. OF PLANNING AND INT'L COOPERATION,NaN,Republic of Yemen!$!RY,RY,Republic of Yemen,"Yemen, Republic of",...,JB,IBRD,Active,N,"{'Percent': 50, 'Name': 'Participation and civ...","[{'code': '57', 'name': 'Participation and civ...","59,57",0,1500000,http://www.worldbank.org/projects/P144665?lang=en
4,{'$oid': '52b213b38594d8a2be17c784'},2014,October,2013-10-31,MINISTRY OF FINANCE,2019-04-30T00:00:00Z,Kingdom of Lesotho!$!LS,LS,Kingdom of Lesotho,Lesotho,...,"FH,YW,YZ",IBRD,Active,N,"{'Percent': 30, 'Name': 'Export development an...","[{'code': '45', 'name': 'Export development an...","41,45",13100000,13100000,http://www.worldbank.org/projects/P144933/seco...


In [109]:
worldbank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 50 columns):
_id                         500 non-null object
approvalfy                  500 non-null int64
board_approval_month        500 non-null object
boardapprovaldate           500 non-null datetime64[ns]
borrower                    485 non-null object
closingdate                 370 non-null object
country_namecode            500 non-null object
countrycode                 500 non-null object
countryname                 500 non-null object
countryshortname            500 non-null object
docty                       446 non-null object
envassesmentcategorycode    430 non-null object
grantamt                    500 non-null int64
ibrdcommamt                 500 non-null int64
id                          500 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
lendinginstr                495 non-null object
lendinginstrtype           

In [108]:
print(worldbank_df['boardapprovaldate'].sort_values().head(1))
print(worldbank_df['boardapprovaldate'].sort_values().tail(1))

499   2012-08-02
Name: boardapprovaldate, dtype: datetime64[ns]
0   2013-11-12
Name: boardapprovaldate, dtype: datetime64[ns]


As this is a subset of the original dataset and the source link doesn't work we'll have to make a few assumptions. The column 'boardapprovaldate' is complete and will be used to mark the date boundaries of the dataset. The column 'closingdate' includes a number of null values, for the purpose of this analysis we'll assume that these projects have not been finalized at this point and lending has not happened.

Nevertheless, this seems like a promising data set of the World Bank stretching from August 2012 to November 2013.

# Countries & Projects

In this first section we'll explore country-specific metrics, specifically the countries with the most projects. 

In [183]:
# Create a list of Country Names
countries_df = worldbank_df.loc[:,['countryname','countrycode','regionname']].drop_duplicates()
# Sort Countries by countrycode
countries_df = countries_df.sort_values(by='countrycode').set_index('countrycode')

# Check Countries DataFrame info
countries_df.info()

countries_df.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 118 entries, 1W to ZW
Data columns (total 2 columns):
countryname    118 non-null object
regionname     118 non-null object
dtypes: object(2)
memory usage: 2.8+ KB


,countryname,regionname
countrycode,,
1W,World,Other
3A,Africa,Africa
4E,East Asia and Pacific,East Asia and Pacific
4P,Pacific Islands,East Asia and Pacific
5M,Middle East and North Africa,Middle East and North Africa
7E,Europe and Central Asia,Europe and Central Asia
8S,South Asia,South Asia
AF,Islamic State of Afghanistan,South Asia
AG,Antigua and Barbuda,Latin America and Caribbean


In [184]:
# Number of Projects assigned to regions rather than countries
regions_codes = ['1W','3A','4E','4P','5M','7E','8S']
print('Number of Projects assigned to Regions ',len(worldbank_df.loc[worldbank_df['countrycode'].isin(regions_codes),:]))

# Remove Regions in countries_df and worldbank_df
worldbank_df_no_regions = worldbank_df.loc[~worldbank_df['countrycode'].isin(regions_codes),:]
countries_df_no_regions = countries_df.loc[~countries_df.index.isin(regions_codes),:]

print('Remaining Projects ',len(worldbank_df_no_regions))
print('Remaining Countries ', len(countries_df_no_regions))

Number of Projects assigned to Regions  25
Remaining Projects  475
Remaining Countries  111


In [185]:
# Percent of Member Countries (189 World Bank Member States) with lending projects
print('Percent of Member Countries with Projects ', round(len(countries_df) / 189 * 100,2),'%')

Percent of Member Countries with Projects  62.43 %


The first exploration of the data set shows that there is a total of 118 unique country codes in the data. We should note that there is a country code for each continent and the world. As these are rather regions than countries and due to a lacking description of how this data could be interpreted, I've decided to remove the region columns as they only make up 25 of the 500 projects. 

This leaves us with 111 actual countries, which is significantly less then the number of recognized countries by the United Nations (195), if we take the number of countries that are members of the Worldbank (189) for comparison, that still leaves us with 71 countries that have no official lending project with Worldbank. On the other hand almost 62.4% have gotten approved for a lending project between Summer 2012 and Winter 2013.

In [186]:
# Group by Countries and Count projects
by_country = worldbank_df_no_regions.groupby('countrycode')['countrycode']
projects_df = by_country.count().sort_index().to_frame(name='projects')
projects_df.head()

,projects
countrycode,
AF,6
AG,1
AL,4
AM,8
AO,1


In [192]:
# Concatenate Countries and Projects and sort by project count descending
countries_and_projects_df = pd.concat([countries_df_no_regions,projects_df],axis='columns').sort_values(by='projects', ascending=False)

# Show Top 10 Countries by Project Count
countries_and_projects_df.head(10)

,countryname,regionname,projects
countrycode,,,
CN,People's Republic of China,East Asia and Pacific,19
ID,Republic of Indonesia,East Asia and Pacific,19
VN,Socialist Republic of Vietnam,East Asia and Pacific,17
IN,Republic of India,South Asia,16
RY,Republic of Yemen,Middle East and North Africa,13
BD,People's Republic of Bangladesh,South Asia,12
NP,Nepal,South Asia,12
MA,Kingdom of Morocco,Middle East and North Africa,12
MZ,Republic of Mozambique,Africa,11


In [195]:
# Sum of Projects for Top 10 Countries
print('Total Number of Projects for the Top 10 ', countries_and_projects_df['projects'].head(10).sum())

Total Number of Projects for the Top 10  140


The top 3 contenders are all from the East Asia and Pacific region. The top 10 countries with the most projects are exclusively coming from Asia and Africa. However, it would be interesting to see if this trend is pervasive across the entire dataset and what type of projects these are.